In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0
)

### Project 3-1. 콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

# 1단계: 더미 분쟁사례 데이터 (5개)
documents = [
    Document(page_content="【사건개요】온라인 게임 아이템 사라짐. 【쟁점사항】시스템 오류로 인한 데이터 손실. 【처리경위】복구 요청 → 거절. 【처리결과】복구 불가, 게임사 귀책 없음."),
    Document(page_content="【사건개요】인터넷 강의 해지 위약금 분쟁. 【쟁점사항】과도한 위약금 요구. 【처리경위】수강 중 해지 요청. 【처리결과】위약금 일부 감경, 계약자 일부 환불 인정."),
    Document(page_content="【사건개요】무료체험 후 자동 결제. 【쟁점사항】환불 가능 여부. 【처리경위】자동 유료 전환, 환불 요청. 【처리결과】서비스 안내 미흡, 일부 환불 인정."),
    Document(page_content="【사건개요】미성년자 결제. 【쟁점사항】부모 동의 없는 아이템 구매. 【처리경위】환불 요청. 【처리결과】부모 동의 없으면 환불 가능."),
    Document(page_content="【사건개요】광고와 다르게 제공된 온라인 교육 서비스. 【쟁점사항】계약 해지 가능 여부. 【처리경위】서비스 불만족, 계약 해지 요청. 【처리결과】계약 해지 인정.")
]

# 1단계: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = text_splitter.split_documents(documents)

# 2단계: 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536)

# 3단계: 벡터스토어 구축
vectorstore = FAISS.from_documents(chunks, embeddings)

# 4단계: 검색기
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# 5단계: LLM 설정
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

# 6단계: 법률 자문 프롬프트
prompt_template = """당신은 콘텐츠 분야 전문 법률 자문사입니다.

관련 분쟁사례: {context}
상담 내용: {question}

답변 가이드라인:
1. 사례를 근거로 답변하세요
2. 관련 법령을 명시하세요  
3. 단계별 해결방안을 제시하세요
4. 유사 사례를 참조하세요
5. 없는 정보는 "확인할 수 없습니다"라고 하세요
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# 7단계: QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 8단계: 테스트 질문
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

# 9단계: 실행 및 결과 출력
for q in test_questions:
    print(f"\n질문: {q}")
    result = qa_chain.invoke({"query": q})
    print("\n[답변]")
    print(result["result"])
    print("\n[참조 사례]")
    for doc in result["source_documents"][:2]:
        print(f"- {doc.page_content[:80]}...")


질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[답변]
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 해결 방안을 다음과 같이 제시할 수 있습니다.

1. **사례 근거**:
   - 앞서 제시된 분쟁 사례 중 "온라인 게임 아이템 사라짐" 사건에서는 시스템 오류로 인한 데이터 손실에 대해 게임사의 귀책이 없다는 결론이 나왔습니다. 이는 게임사의 약관에 시스템 오류로 인한 손실에 대한 책임을 면제하는 조항이 포함되어 있을 가능성이 높습니다.

2. **관련 법령**:
   - 전자상거래 등에서의 소비자 보호에 관한 법률에 따르면, 사업자는 소비자에게 중요한 정보를 명확히 제공해야 하며, 계약의 이행 과정에서 발생하는 문제에 대해 책임을 질 의무가 있습니다. 그러나 약관에 명시된 면책 조항이 법적으로 유효하다면, 게임사의 책임이 제한될 수 있습니다.

3. **단계별 해결방안**:
   - **약관 검토**: 게임사의 이용 약관을 검토하여 시스템 오류로 인한 손실에 대한 면책 조항이 있는지 확인합니다.
   - **증거 수집**: 아이템 손실과 관련된 모든 증거(스크린샷, 오류 메시지, 고객센터와의 대화 기록 등)를 수집합니다.
   - **고객센터 재접촉**: 수집한 증거를 바탕으로 고객센터에 다시 복구 요청을 하며, 약관의 불공정성을 주장할 수 있습니다.
   - **소비자 보호원 상담**: 한국소비자원 등 관련 기관에 상담을 요청하여 추가적인 법적 조언을 받습니다.
   - **법적 대응 고려**: 위의 방법으로 해결되지 않을 경우, 법적 대응을 고려할 수 있습니다. 이때, 변호사와 상담하여 소송 가능성을 평가합니다.

4. **유사 사례 참조**:
   - "광고와 다르게 제공된 온라인 교육 서비스" 사례에서는 계약 해지가 인정되었습니다. 이는 제공된 서비스가 광고와 다를 경우 소비자가 계약을 해지할 수 있음을 보여줍니다. 